# Training a Simple GAN Model for Sentence Embeddings

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter  # to print to tensorboard
import pandas as pd
import ast
import numpy as np
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import asarray
from numpy.random import randn
from numpy.random import randint
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv1D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from keras.layers import Lambda
from keras.layers import Activation
from matplotlib import pyplot as plt
from keras import backend
from keras.models import Sequential
from sklearn.model_selection import train_test_split


MAX_LENGTH = 768


class Discriminator(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.disc = nn.Sequential(
            nn.Linear(in_features, 128),
            nn.LeakyReLU(0.01),
            nn.Linear(128, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        return self.disc(x)

class Generator(nn.Module):
    def __init__(self, z_dim, emb_dim):
        super().__init__()
        self.gen = nn.Sequential(
            nn.Linear(z_dim, 256),
            nn.LeakyReLU(0.01),
            nn.Linear(256, emb_dim),
            nn.Tanh(),  # Assuming you want to normalize the outputs
        )

    def forward(self, x):
        return self.gen(x)


class CustomDataset(Dataset):
    def __init__(self, embeddings):
        self.embeddings = embeddings

    def __len__(self):
        return len(self.embeddings)

    def __getitem__(self, idx):
        return self.embeddings[idx]



# Hyperparameters etc.
device = "cuda" if torch.cuda.is_available() else "cpu"
lr = 3e-4
z_dim = 64
embed_dim = MAX_LENGTH  # 784
batch_size = 32
num_epochs = 50

disc = Discriminator(embed_dim).to(device)
gen = Generator(z_dim, embed_dim).to(device)
fixed_noise = torch.randn((batch_size, z_dim)).to(device)
transforms = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,)),
    ]
)


2024-03-29 17:02:31.480712: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Preprocessing Data

In [2]:
#Limit number of rows for experimentation
df = pd.read_csv('author_csv.csv')
num_rows = len(df)
df = df[:num_rows]
df

,Unnamed: 0,author_labels,cls_tokens
0,0,1,"[0.278237104415893, -0.33750003576278603, 0.84..."
1,1,1,"[-0.13946822285652102, 0.093057677149772, 0.73..."
2,2,1,"[-1.493581295013427, 0.748962104320526, 1.0086..."
3,3,1,"[-0.9141901135444641, 0.6804959774017331, 0.90..."
4,4,1,"[0.040547348558902005, 0.08085644245147701, 1...."
...,...,...,...
32592,32592,0,"[0.036508537828922, -0.9830706119537351, 0.194..."
32593,32593,0,"[-0.49513417482376104, -0.42453348636627203, 0..."
32594,32594,0,"[-0.6720252633094781, -0.37544131278991705, 2...."
32595,32595,0,"[0.153745874762535, -0.533583104610443, -0.371..."


In [3]:
df['author_labels'].value_counts()

author_labels
1    27903
0     2352
2     2342
Name: count, dtype: int64

In [4]:
# Load the text lines

embeddings = df['cls_tokens']

#Turn EagerTensors list to Normal Tensors list
embeddings_pytorch = [torch.tensor(np.array(ast.literal_eval(e)), dtype=torch.float32) for e in embeddings]

# Convert list of tensors to a single tensor
embeddings_tensor = torch.stack(embeddings_pytorch).squeeze(1)  # Adjust dimensions as needed

embeddings_tensor

tensor([[ 0.2782, -0.3375,  0.8478,  ..., -0.7637, -0.5338,  0.3607],
        [-0.1395,  0.0931,  0.7390,  ...,  0.8308, -0.3973,  1.1339],
        [-1.4936,  0.7490,  1.0087,  ...,  1.5033, -1.2829, -0.5658],
        ...,
        [-0.6720, -0.3754,  2.1211,  ...,  1.0377, -0.6048, -0.7254],
        [ 0.1537, -0.5336, -0.3715,  ...,  1.6371,  0.6499, -0.5228],
        [-0.7284, -0.3828, -0.2024,  ...,  1.4613,  0.1921,  1.5803]])

In [5]:
embeddings_tensor.shape

torch.Size([32597, 768])

In [6]:
embeddings = embeddings_tensor


In [7]:
#Initialize Dataset
embeddings = embeddings_tensor 

# Instantiate the custom dataset
dataset = CustomDataset(embeddings)

# Training the SGAN Model

In [8]:
from keras.layers import Layer
from keras import backend as K

#Ended up not using either of these custom layers, so ignore for now
class CustomActivationLayer(Layer):
    def call(self, inputs):
        logexpsum = K.sum(K.exp(inputs), axis=-1, keepdims=True)
        result = logexpsum / (logexpsum + 1.0)
        return result

    def compute_output_shape(self, input_shape):
        # Assuming your layer does not change the input shape
        return input_shape

    


In [9]:
import tensorflow as tf

class CustomActivationLayer(Layer):
    def call(self, inputs):
        logexpsum = tf.reduce_sum(tf.exp(inputs), axis=-1, keepdims=True)
        result = logexpsum / (logexpsum + 1.0)
        return result

    def compute_output_shape(self, input_shape):
        # Assuming your layer does not change the input shape
        return input_shape


In [10]:
X = embeddings_tensor
y = df['author_labels'].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=6)
print(X_test.shape)
print(y_test.shape)

torch.Size([8150, 768])
(8150,)


The next code block contains all the helper functions and the training script. The main functions to look at are the 
define_discriminator, define_generator, and define_gan functions. They contain the model architecture for the discriminator and generator functions.

In [15]:
lr = 0.0002
n_samples = 1000
n_epochs = 2
loss_var_threshold = 0.01

# define the standalone supervised and unsupervised discriminator models
def define_discriminator(in_shape=(MAX_LENGTH,), n_classes=df['author_labels'].nunique()):
    
  # Embedding input
  in_image = Input(shape=in_shape)

  # downsample
  fe = Dense(units=100, activation='sigmoid')(in_image)
  fe = LeakyReLU(negative_slope=0.2)(fe)

  # downsample
  fe = Dense(units=100, activation='sigmoid')(in_image)
  fe = LeakyReLU(negative_slope=0.2)(fe)
    
  # dropout
  fe = Dropout(0.2)(fe)

  # output layer nodes
  fe = Dense(n_classes)(fe)
    
  # supervised output
  c_out_layer = Activation('softmax')(fe)
    
  # define and compile supervised discriminator model
  c_model = Model(in_image, c_out_layer)
  c_model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=lr, beta_1=0.5), metrics=['accuracy'])
    
  # unsupervised output
  d_out_layer = Dense(units = 1, activation = 'sigmoid')(fe)
    
  # define and compile unsupervised discriminator model
  d_model = Model(in_image, d_out_layer)
  d_model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=lr, beta_1=0.5), metrics = ['accuracy'])
  d_model.trainable = True
  c_model.trainable = True
    
  return d_model, c_model

# define the standalone generator model
def define_generator(latent_dim, n_outputs=MAX_LENGTH):
	model = Sequential()
	model.add(Dense(200, activation='sigmoid', kernel_initializer='he_uniform', input_dim=latent_dim))
	model.add(Dense(100, activation='sigmoid'))
	model.add(Dense(n_outputs, activation='relu'))
	return model

# define the combined generator and discriminator model, for updating the generator
def define_gan(generator, discriminator):
    # Ensure the discriminator's parameters are not trainable in the combined model
    discriminator.trainable = False
    
    # Create a new input layer for the GAN (noise sample)
    gan_input = Input(shape=(latent_dim,))
    
    # Output of the generator
    generator_output = generator(gan_input)
    
    # Output of the discriminator (takes generator's output as input)
    gan_output = discriminator(generator_output)
    
    # Define the GAN model
    gan = Model(gan_input, gan_output)
    
    # Compile the GAN model
    optimizer = Adam(learning_rate=lr)
    gan.compile(optimizer=optimizer, loss='binary_crossentropy')
    
    return gan


# load the embeddings and classifications
def load_real_samples(X,y):
	print(X.shape, y.shape)
	return [X, y]

# select a supervised subset of the dataset, ensures classes are balanced
def select_supervised_samples(dataset, n_samples=n_samples, n_classes=df['author_labels'].nunique()):
    X, y = dataset
    n_per_class = int(n_samples / n_classes)
    X_samples = []
    y_samples = []

    
    for class_index in range(n_classes):
        
        # Find the indices of all samples belonging to the current class
        class_indices = np.where(y == class_index)[0]
        
        # Randomly choose n_per_class indices for this class
        selected_indices = np.random.choice(class_indices, n_per_class, replace=False)
        # Append the selected samples to the lists
        X_samples.append(X[selected_indices])
        y_samples.append(y[selected_indices])
    
    # Concatenate all selected samples
    X_samples = np.concatenate(X_samples, axis=0)
    y_samples = np.concatenate(y_samples, axis=0)
    return X_samples, y_samples

    
def generate_real_samples(dataset, n_samples):
    features, labels = dataset
    # Generate random indices
    indices = np.random.choice(features.shape[0], n_samples, replace=False)
    # Select a random subset of features and labels using the indices
    X = features[indices]
    labels = labels[indices]
    # Generate class labels (assuming you want all ones for real samples)
    y = np.ones((n_samples, 1))
    return [X, labels], y


# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space
	z_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	z_input = z_input.reshape(n_samples, latent_dim)
	return z_input

# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
	# generate points in latent space
	z_input = generate_latent_points(latent_dim, n_samples)
	# predict outputs
	images = generator.predict(z_input)
	# create class labels
	y = zeros((n_samples, 1))
	return images, y

# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, c_model, latent_dim, dataset, acc_list, n_samples=100):
    
	# prepare fake examples
	X, _ = generate_fake_samples(g_model, latent_dim, n_samples)
    
	# scale from [-1,1] to [0,1]
	X = (X + 1) / 2.0
    
	# evaluate the classifier model
	X, y = dataset
	_, acc = c_model.evaluate(X, y, verbose=0)
	acc_list.append(acc)
	print('Classifier Accuracy: %.3f%%' % (acc * 100))
    
	# save the generator model
	filename2 = 'g_model_%04d.h5' % (step+1)
	g_model.save(filename2)
    
	# save the classifier model
	filename3 = 'c_model_%04d.h5' % (step+1)
	c_model.save(filename3)
	print('>Saved:  %s and %s' % (filename2, filename3))

# train the generator and discriminator
def train(g_model, d_model, c_model, gan_model, dataset, latent_dim, acc_list, n_epochs=n_epochs, n_batch=100):
    print(f'Latent dimensions: {latent_dim}\n-------------')
    # Initialize lists to track losses
    c_losses, d_losses, g_losses = [], [], []
    loss_variance_threshold = loss_var_threshold  # Set a threshold for the variance
    min_epochs = 5  # Set a minimum number of epochs to prevent stopping too early
    
    # select supervised dataset
    X_sup, y_sup = select_supervised_samples(dataset)
    print(X_sup.shape, y_sup.shape)
    
    # calculate the number of batches per training epoch
    bat_per_epo = int(dataset[0].shape[0] / n_batch)
    
    # calculate the number of training iterations
    n_steps = bat_per_epo * n_epochs
    
    # calculate the size of half a batch of samples
    half_batch = int(n_batch / 2)
    print('n_epochs=%d, n_batch=%d, 1/2=%d, b/e=%d, steps=%d' % (n_epochs, n_batch, half_batch, bat_per_epo, n_steps))
    
    # manually enumerate epochs
    for i in range(n_steps):
        
        # update supervised discriminator (c)
        [Xsup_real, ysup_real], _ = generate_real_samples([X_sup, y_sup], n_batch)
        c_loss, c_acc = c_model.train_on_batch(Xsup_real, ysup_real)
        
        # update unsupervised discriminator (d) on real samples
        [X_real, _], y_real = generate_real_samples(dataset, n_batch)
        d_loss1, d_acc1 = d_model.train_on_batch(X_real, y_real)  # Capture loss and accuracy
        
        # update unsupervised discriminator (d) on fake samples
        X_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_batch)
        d_loss2, d_acc2 = d_model.train_on_batch(X_fake, y_fake)  # Capture loss and accuracy

        
        # update generator (g)
        X_gan, y_gan = generate_latent_points(latent_dim, n_batch), ones((n_batch, 1))
        g_loss = gan_model.train_on_batch(X_gan, y_gan)
        
        # summarize loss on this batch
        print('>%d, c(loss and accuracy)[%.3f,%.0f], d(loss on real and fake)[%.3f,%.3f], d(acc on real and fake)[%.3f,%.3f], g(loss)[%.3f]' % (i+1, c_loss, c_acc*100, d_loss1, d_loss2, d_acc1 * 100, d_acc2 * 100, g_loss))
        
        # evaluate the model performance every so often
        if (i+1) % (100) == 0:
            summarize_performance(i, g_model, c_model, latent_dim, dataset, acc_list)
    
        # At the end of each epoch (or defined interval) in training loop:
        c_losses.append(c_loss)
        d_losses.append((d_loss1 + d_loss2) / 2)
        g_losses.append(g_loss)
        
        # Calculate the variance of the last N losses to check for stability
        if len(c_losses) > min_epochs * (i + 1):  # Ensure we have enough data to make a meaningful decision
            recent_c_var = np.var(c_losses[-min_epochs:])
            recent_d_var = np.var(d_losses[-min_epochs:])
            recent_g_var = np.var(g_losses[-min_epochs:])
            
            # Check if all losses have stabilized
            if recent_c_var < loss_variance_threshold and recent_d_var < loss_variance_threshold and recent_g_var < loss_variance_threshold:
                print(f"Stopping training at epoch {i+1} due to stabilized losses.")
                break



In [12]:
def def_models(latent_dim):
    d_model, c_model = define_discriminator()
    g_model = define_generator(latent_dim)
    gan_model = define_gan(g_model, d_model)
    return d_model, c_model, g_model, gan_model

def train_script(latent_dim):
    d_model, c_model, g_model, gan_model = def_models(latent_dim)

    train(g_model, d_model, c_model, gan_model, dataset, latent_dim, acc_list)


In [16]:
#accuracy list for each epochs
acc_list = []
# size of the latent space
latent_dim = 100
# create the discriminator models
d_model, c_model = define_discriminator()

# create the generator
g_model = define_generator(latent_dim)

# create the gan
gan_model = define_gan(g_model, d_model)

# load embedding data
dataset = load_real_samples(X_train,y_train)
c_model.summary() #Summary of supervised discriminator

d_model.summary() #Summary of unsupervised discriminator

torch.Size([24447, 768]) (24447,)


Model: "functional_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 768)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 100)            │        76,900 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_3 (LeakyReLU)       │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 3)              │           303 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 3)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 77,203 (301.57 KB)

 Trainable params: 0 (0.00 B)

...

Model: "functional_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 768)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 100)            │        76,900 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_3 (LeakyReLU)       │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 3)              │           303 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 1)              │             4 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 77,207 (301.59 KB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 77,207 (301.59 KB)

In [17]:
train(g_model, d_model, c_model, gan_model, dataset, latent_dim, acc_list)

Latent dimensions: 100
-------------
(999, 768) (999,)
n_epochs=2, n_batch=100, 1/2=50, b/e=244, steps=488
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
>1, c(loss and accuracy)[1.263,33], d(loss on real and fake)[0.968,0.753], d(acc on real and fake)[3.000,50.500], g(loss)[0.854]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
>2, c(loss and accuracy)[1.210,35], d(loss on real and fake)[0.817,0.756], d(acc on real and fake)[35.000,48.000], g(loss)[0.865]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
>3, c(loss and accuracy)[1.205,38], d(loss on real and fake)[0.799,0.763], d(acc on real and fake)[39.000,47.667], g(loss)[0.859]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
>4, c(loss and accuracy)[1.223,35], d(loss on real and fake)[0.790,0.766], d(acc on real and fake)[41.714,46.875], g(loss)[0.845]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
>5, c(loss and accuracy)[1.235,35], d(loss on real and fake)[0.785,0.768], d(acc on real and fake)[42.444,46.500], g(loss)[0.831]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
>6, c(loss and a

Classifier Accuracy: 73.952%
>Saved:  g_model_0100.h5 and c_model_0100.h5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
>101, c(loss and accuracy)[0.839,74], d(loss on real and fake)[1.166,1.170], d(acc on real and fake)[7.090,7.054], g(loss)[0.349]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
>102, c(loss and accuracy)[0.840,74], d(loss on real and fake)[1.169,1.173], d(acc on real and fake)[7.044,7.010], g(loss)[0.347]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
>103, c(loss and accuracy)[0.840,73], d(loss on real and fake)[1.172,1.177], d(acc on real and fake)[7.010,6.976], g(loss)[0.345]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
>104, c(loss and accuracy)[0.841,73], d(loss on real and fake)[1.176,1.180], d(acc on real and fake)[6.971,6.938], g(loss)[0.343]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
>105, c(loss and accuracy)[0.841,73], d(loss on real and fake)[1.179,1.183], d(acc on real and fake)[6.943,6.910], g(loss)[0.341]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
>106, c(loss and accuracy)[0.842,73], d(loss on 

Classifier Accuracy: 73.952%
>Saved:  g_model_0200.h5 and c_model_0200.h5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
>201, c(loss and accuracy)[0.839,74], d(loss on real and fake)[1.391,1.394], d(acc on real and fake)[4.810,4.799], g(loss)[0.231]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
>202, c(loss and accuracy)[0.840,74], d(loss on real and fake)[1.393,1.395], d(acc on real and fake)[4.799,4.787], g(loss)[0.230]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
>203, c(loss and accuracy)[0.840,73], d(loss on real and fake)[1.394,1.397], d(acc on real and fake)[4.793,4.781], g(loss)[0.230]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
>204, c(loss and accuracy)[0.841,73], d(loss on real and fake)[1.396,1.399], d(acc on real and fake)[4.784,4.772], g(loss)[0.229]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
>205, c(loss and accuracy)[0.841,73], d(loss on real and fake)[1.398,1.400], d(acc on real and fake)[4.775,4.763], g(loss)[0.228]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
>206, c(loss and accuracy)[0.842,73], d(loss on 

Classifier Accuracy: 73.952%
>Saved:  g_model_0300.h5 and c_model_0300.h5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
>301, c(loss and accuracy)[0.839,74], d(loss on real and fake)[1.516,1.518], d(acc on real and fake)[4.115,4.108], g(loss)[0.181]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
>302, c(loss and accuracy)[0.840,74], d(loss on real and fake)[1.517,1.519], d(acc on real and fake)[4.109,4.103], g(loss)[0.181]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
>303, c(loss and accuracy)[0.840,74], d(loss on real and fake)[1.518,1.520], d(acc on real and fake)[4.106,4.099], g(loss)[0.181]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
>304, c(loss and accuracy)[0.841,73], d(loss on real and fake)[1.519,1.521], d(acc on real and fake)[4.105,4.099], g(loss)[0.180]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
>305, c(loss and accuracy)[0.841,73], d(loss on real and fake)[1.520,1.522], d(acc on real and fake)[4.097,4.090], g(loss)[0.180]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
>306, c(loss and accuracy)[0.842,73], d(loss on 

Classifier Accuracy: 73.952%
>Saved:  g_model_0400.h5 and c_model_0400.h5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
>401, c(loss and accuracy)[0.839,74], d(loss on real and fake)[1.597,1.599], d(acc on real and fake)[3.775,3.771], g(loss)[0.154]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
>402, c(loss and accuracy)[0.840,74], d(loss on real and fake)[1.598,1.599], d(acc on real and fake)[3.770,3.765], g(loss)[0.153]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
>403, c(loss and accuracy)[0.840,74], d(loss on real and fake)[1.599,1.600], d(acc on real and fake)[3.765,3.761], g(loss)[0.153]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
>404, c(loss and accuracy)[0.841,73], d(loss on real and fake)[1.599,1.601], d(acc on real and fake)[3.762,3.757], g(loss)[0.153]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
>405, c(loss and accuracy)[0.841,73], d(loss on real and fake)[1.600,1.601], d(acc on real and fake)[3.764,3.759], g(loss)[0.153]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
>406, c(loss and accuracy)[0.842,73], d(loss on 

In [18]:
test_loss, test_accuracy = c_model.evaluate(X_test, y_test, verbose=0)
print('Test Accuracy for multiclass classification model: %.3f%%' % (test_accuracy * 100))

Test Accuracy for multiclass classification model: 73.607%


It looks like the multiclass discriminator model performed terribly on the test set. We will now test our binary discriminator that tests our model's ability to distinguish real from fake.

In [19]:

#Since everything in our X_test and y_test are "real" samples, 
#let's add some fake samples so there is a 50/50 split between real and fake
fake_embeddings, classification = generate_fake_samples(g_model, latent_dim, len(X_test))

255/255 ━━━━━━━━━━━━━━━━━━━━ 0s 937us/step


In [20]:
X_test_with_fakes = np.concatenate((X_test, fake_embeddings), axis = 0)

In [21]:
y_test_with_fakes = np.concatenate((np.ones(len(X_test)), classification.flatten()), axis = 0)

In [22]:
# Generate shuffled indices
indices = tf.range(start=0, limit=tf.shape(X_test_with_fakes)[0], dtype=tf.int32)
shuffled_indices = tf.random.shuffle(indices)

# Reorder both X and y using the shuffled indices
X_test_with_fakes_shuffled = tf.gather(X_test_with_fakes, shuffled_indices)
y_test_with_fakes_shuffled = tf.gather(y_test_with_fakes, shuffled_indices)

In [23]:
test_loss, test_accuracy = d_model.evaluate(X_test_with_fakes_shuffled, y_test_with_fakes_shuffled, verbose=0)
print('Test Accuracy for binary discriminator: %.3f%%' % (test_accuracy * 100))

Test Accuracy for binary discriminator: 0.055%


It looks like the binary discriminator didn't perform anywhere above the baseline.

### Grid Search with Latent Dimensions

In [24]:
latent_dims = [100, 200, 300 , 400, 500]

for latent_dim in latent_dims:
    train_script(latent_dim)

Latent dimensions: 100
-------------
(999, 768) (999,)
n_epochs=2, n_batch=100, 1/2=50, b/e=244, steps=488
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
>1, c(loss and accuracy)[1.165,34], d(loss on real and fake)[0.513,0.759], d(acc on real and fake)[80.000,45.500], g(loss)[0.487]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
>2, c(loss and accuracy)[1.181,32], d(loss on real and fake)[0.677,0.764], d(acc on real and fake)[56.667,45.250], g(loss)[0.468]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
>3, c(loss and accuracy)[1.195,32], d(loss on real and fake)[0.722,0.777], d(acc on real and fake)[52.800,45.167], g(loss)[0.466]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
>4, c(loss and accuracy)[1.179,33], d(loss on real and fake)[0.739,0.787], d(acc on real and fake)[50.286,45.000], g(loss)[0.451]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
>5, c(loss and accuracy)[1.182,32], d(loss on real and fake)[0.755,0.797], d(acc on real and fake)[49.222,44.700], g(loss)[0.444]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
>6, c(loss and 

Classifier Accuracy: 73.216%
>Saved:  g_model_0100.h5 and c_model_0100.h5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
>101, c(loss and accuracy)[0.917,73], d(loss on real and fake)[1.628,1.639], d(acc on real and fake)[41.751,41.545], g(loss)[0.111]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
>102, c(loss and accuracy)[0.918,73], d(loss on real and fake)[1.634,1.645], d(acc on real and fake)[41.724,41.520], g(loss)[0.110]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
>103, c(loss and accuracy)[0.918,73], d(loss on real and fake)[1.640,1.651], d(acc on real and fake)[41.746,41.544], g(loss)[0.109]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
>104, c(loss and accuracy)[0.919,73], d(loss on real and fake)[1.646,1.657], d(acc on real and fake)[41.729,41.529], g(loss)[0.108]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
>105, c(loss and accuracy)[0.919,72], d(loss on real and fake)[1.651,1.662], d(acc on real and fake)[41.732,41.533], g(loss)[0.107]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
>106, c(loss and accuracy)[0.919,72], 

Classifier Accuracy: 73.216%
>Saved:  g_model_0200.h5 and c_model_0200.h5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
>201, c(loss and accuracy)[0.917,73], d(loss on real and fake)[2.028,2.034], d(acc on real and fake)[41.716,41.612], g(loss)[0.063]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
>202, c(loss and accuracy)[0.918,73], d(loss on real and fake)[2.031,2.037], d(acc on real and fake)[41.702,41.599], g(loss)[0.063]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
>203, c(loss and accuracy)[0.918,73], d(loss on real and fake)[2.034,2.040], d(acc on real and fake)[41.696,41.594], g(loss)[0.063]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
>204, c(loss and accuracy)[0.918,73], d(loss on real and fake)[2.037,2.043], d(acc on real and fake)[41.717,41.615], g(loss)[0.062]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
>205, c(loss and accuracy)[0.919,72], d(loss on real and fake)[2.039,2.046], d(acc on real and fake)[41.714,41.612], g(loss)[0.062]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
>206, c(loss and accuracy)[0.919,72], 

Classifier Accuracy: 73.216%
>Saved:  g_model_0300.h5 and c_model_0300.h5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
>301, c(loss and accuracy)[0.917,73], d(loss on real and fake)[2.264,2.269], d(acc on real and fake)[41.644,41.575], g(loss)[0.045]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
>302, c(loss and accuracy)[0.918,73], d(loss on real and fake)[2.266,2.271], d(acc on real and fake)[41.633,41.565], g(loss)[0.045]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
>303, c(loss and accuracy)[0.918,73], d(loss on real and fake)[2.268,2.273], d(acc on real and fake)[41.638,41.569], g(loss)[0.044]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
>304, c(loss and accuracy)[0.919,72], d(loss on real and fake)[2.270,2.275], d(acc on real and fake)[41.626,41.558], g(loss)[0.044]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
>305, c(loss and accuracy)[0.919,72], d(loss on real and fake)[2.272,2.277], d(acc on real and fake)[41.621,41.552], g(loss)[0.044]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
>306, c(loss and accuracy)[0.919,72], 

Classifier Accuracy: 73.216%
>Saved:  g_model_0400.h5 and c_model_0400.h5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
>401, c(loss and accuracy)[0.917,73], d(loss on real and fake)[2.431,2.435], d(acc on real and fake)[41.588,41.536], g(loss)[0.035]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
>402, c(loss and accuracy)[0.918,73], d(loss on real and fake)[2.432,2.436], d(acc on real and fake)[41.592,41.540], g(loss)[0.035]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
>403, c(loss and accuracy)[0.918,73], d(loss on real and fake)[2.433,2.437], d(acc on real and fake)[41.594,41.542], g(loss)[0.035]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
>404, c(loss and accuracy)[0.918,73], d(loss on real and fake)[2.435,2.438], d(acc on real and fake)[41.594,41.542], g(loss)[0.035]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
>405, c(loss and accuracy)[0.919,72], d(loss on real and fake)[2.436,2.440], d(acc on real and fake)[41.598,41.547], g(loss)[0.035]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
>406, c(loss and accuracy)[0.919,72], 

...

Classifier Accuracy: 9.752%
>Saved:  g_model_0400.h5 and c_model_0400.h5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
>401, c(loss and accuracy)[1.345,10], d(loss on real and fake)[2.076,2.079], d(acc on real and fake)[49.991,49.929], g(loss)[0.037]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
>402, c(loss and accuracy)[1.344,10], d(loss on real and fake)[2.077,2.080], d(acc on real and fake)[49.991,49.929], g(loss)[0.037]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
>403, c(loss and accuracy)[1.344,10], d(loss on real and fake)[2.078,2.081], d(acc on real and fake)[49.991,49.929], g(loss)[0.037]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
>404, c(loss and accuracy)[1.344,10], d(loss on real and fake)[2.079,2.083], d(acc on real and fake)[49.991,49.929], g(loss)[0.037]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
>405, c(loss and accuracy)[1.344,10], d(loss on real and fake)[2.080,2.084], d(acc on real and fake)[49.991,49.930], g(loss)[0.037]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
>406, c(loss and accuracy)[1.343,10], d

Classifier Accuracy: 66.049%
>Saved:  g_model_0100.h5 and c_model_0100.h5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
>101, c(loss and accuracy)[0.839,66], d(loss on real and fake)[1.251,1.258], d(acc on real and fake)[25.388,25.262], g(loss)[0.227]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
>102, c(loss and accuracy)[0.840,66], d(loss on real and fake)[1.255,1.261], d(acc on real and fake)[25.384,25.260], g(loss)[0.226]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
>103, c(loss and accuracy)[0.841,66], d(loss on real and fake)[1.259,1.265], d(acc on real and fake)[25.371,25.248], g(loss)[0.224]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
>104, c(loss and accuracy)[0.842,66], d(loss on real and fake)[1.263,1.269], d(acc on real and fake)[25.324,25.202], g(loss)[0.223]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
>105, c(loss and accuracy)[0.843,65], d(loss on real and fake)[1.266,1.273], d(acc on real and fake)[25.316,25.195], g(loss)[0.222]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
>106, c(loss and accuracy)[0.844,65], 

Classifier Accuracy: 66.049%
>Saved:  g_model_0200.h5 and c_model_0200.h5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
>201, c(loss and accuracy)[0.839,66], d(loss on real and fake)[1.526,1.530], d(acc on real and fake)[25.219,25.157], g(loss)[0.144]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
>202, c(loss and accuracy)[0.840,66], d(loss on real and fake)[1.528,1.532], d(acc on real and fake)[25.228,25.166], g(loss)[0.143]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
>203, c(loss and accuracy)[0.841,66], d(loss on real and fake)[1.530,1.534], d(acc on real and fake)[25.210,25.148], g(loss)[0.143]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
>204, c(loss and accuracy)[0.842,65], d(loss on real and fake)[1.532,1.536], d(acc on real and fake)[25.214,25.152], g(loss)[0.142]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
>205, c(loss and accuracy)[0.843,65], d(loss on real and fake)[1.534,1.538], d(acc on real and fake)[25.205,25.144], g(loss)[0.142]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
>206, c(loss and accuracy)[0.843,65], 

Classifier Accuracy: 66.049%
>Saved:  g_model_0300.h5 and c_model_0300.h5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
>301, c(loss and accuracy)[0.839,66], d(loss on real and fake)[1.683,1.686], d(acc on real and fake)[25.201,25.159], g(loss)[0.109]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
>302, c(loss and accuracy)[0.840,66], d(loss on real and fake)[1.685,1.688], d(acc on real and fake)[25.211,25.169], g(loss)[0.109]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
>303, c(loss and accuracy)[0.841,66], d(loss on real and fake)[1.686,1.689], d(acc on real and fake)[25.200,25.158], g(loss)[0.109]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
>304, c(loss and accuracy)[0.842,66], d(loss on real and fake)[1.687,1.690], d(acc on real and fake)[25.193,25.151], g(loss)[0.108]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
>305, c(loss and accuracy)[0.843,65], d(loss on real and fake)[1.688,1.691], d(acc on real and fake)[25.189,25.148], g(loss)[0.108]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
>306, c(loss and accuracy)[0.843,65], 

Classifier Accuracy: 66.049%
>Saved:  g_model_0400.h5 and c_model_0400.h5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
>401, c(loss and accuracy)[0.839,66], d(loss on real and fake)[1.791,1.794], d(acc on real and fake)[25.065,25.034], g(loss)[0.090]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
>402, c(loss and accuracy)[0.840,66], d(loss on real and fake)[1.792,1.795], d(acc on real and fake)[25.066,25.035], g(loss)[0.090]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
>403, c(loss and accuracy)[0.841,66], d(loss on real and fake)[1.793,1.795], d(acc on real and fake)[25.070,25.038], g(loss)[0.089]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
>404, c(loss and accuracy)[0.841,66], d(loss on real and fake)[1.794,1.796], d(acc on real and fake)[25.067,25.036], g(loss)[0.089]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
>405, c(loss and accuracy)[0.842,65], d(loss on real and fake)[1.795,1.797], d(acc on real and fake)[25.069,25.038], g(loss)[0.089]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
>406, c(loss and accuracy)[0.843,65], 

In [25]:
test_loss, test_accuracy = c_model.evaluate(X_test, y_test, verbose=0)
print('Test Accuracy for multiclass classification model: %.3f%%' % (test_accuracy * 100))

Test Accuracy for multiclass classification model: 73.607%
